In [97]:
'''
Analysis on how the ownership of a token holder change
'''

'\nAnalysis on how the ownership of a token holder change\n'

In [98]:
from bs4 import BeautifulSoup
import urllib3
import re
import pandas
import numpy

In [101]:
def html_convert(html, classname):
    soup = BeautifulSoup(html)
    table = soup.find("table", attrs=classname)
    headings = [th.get_text() for th in table.find("tr").find_all("th")]
    for idx in range(len(headings)):
        headings[idx]=str(headings[idx])
    datasets = []
    for row in table.find_all("tr")[1:]:
        dataset = (td.get_text() for td in row.find_all("td"))
        ls=list(dataset)
        datasets.append(ls)

    '''
    Clean the data
    '''
    for idx in range(len(datasets)):
        for idxx in range(len(datasets[idx])):
            tmp=re.sub(r'\([^)]*\)', '', datasets[idx][idxx])
            tmp=tmp.strip()
            tmp=str(tmp)
            datasets[idx][idxx]=tmp

    '''
    Create pandas dataframe and convert it to float
    '''
    df=pandas.DataFrame(datasets, columns=headings)
    df['Quantity (Token)']=df['Quantity (Token)'].astype(numpy.float64)
    df['Percentage']=df['Percentage'].str.strip('%')
    df['Percentage']=df['Percentage'].astype(numpy.float64)/100
    return df

baseUrl='https://etherscan.io/'
funUrl_chart='token/tokenholderchart/'
tokenid='0x86fa049857e0209aa7d9e616f7eb3b3b78ecfdb0'

'''
Get the top 100 owners of the coin
'''
pool=urllib3.PoolManager()
r=pool.request('GET',baseUrl+funUrl_chart+tokenid)
webdata=r.data
df=html_convert(webdata, "table table-hover ")
df

/usr/local/lib/python2.7/site-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


,Rank,Address,Quantity (Token),Percentage
0,1,0xd0a6e6c54dbc68db5db3a091b171a77407ff7ccf,6.611222e+08,0.661122
1,2,0x00000000000000000000000000000000000000b1,1.000000e+08,0.100000
2,3,0x42da8a05cb7ed9a43572b5ba1b8f82a0a6e263dc,3.016035e+07,0.030160
3,4,0xf4b51b14b9ee30dc37ec970b50a486f37686e2a8,1.438695e+07,0.014387
4,5,0x174443351e21d47ed9ab51517a301107d92ede64,1.390607e+07,0.013906
5,6,0xe2dd88165572932944ba72ab6ed143f9a9855a81,4.500000e+06,0.004500
6,7,0xb819dfe8927f1055467c957083a752e2f67e4d65,3.501904e+06,0.003502
7,8,0x0aaca9876ec3fe0245daec6b17c52786800ebb7b,3.286042e+06,0.003286
8,9,0x6dde928226f1bccf0932396c7f8501c61c04577a,3.270733e+06,0.003271
9,10,0x8a8f09adb437b2bfdbb4ae53ad82c7791ef443f1,2.510748e+06,0.002511
